### General Sandbox

In [113]:
import datetime
import random
import pandas as pd
date_set = [datetime.date(2018,1,1) + datetime.timedelta(days=i) for i in range(240)]
user_list = ['User-'+'{0:03}'.format(i) for i in range(200)]
numer_of_observations = 500
df = pd.DataFrame(list(zip(random.choices(user_list,k=numer_of_observations),random.choices(date_set,k=numer_of_observations), random.choices(['on','off'],k=numer_of_observations))), columns=['user','date', 'state'])
#df = df[df['sender_id']!=df['accepter_id']][['date','sender_id','accepter_id']]
#agg1 = {'user':{'users_active_today' : (lambda z : set(z))} }
#df1 = df.groupby('date').agg(agg1)

In [118]:
df[(df['date']==datetime.date.today()) & (df['state']=='on')]

,user,date,state
263,User-082,2018-08-27,on
377,User-193,2018-08-27,on
448,User-134,2018-08-27,on


In [89]:
lst1 = random.choices(user_list,k=numer_of_observations)

In [90]:
lst2 = lst1.copy()

In [101]:
lst1[[lst2[i]==lst1[i] for i in range(len(lst2))].index(False)]

'User-032'

In [2]:
def get_active_users(date):
    s1 = set([date - datetime.timedelta(days=i) for i in range(28)])
    return set(df[df['date'].isin(s1)]['user'])

def get_all_prior_users(date):
    return set(df[df['date'] < date]['user'])

In [3]:
df2 = {date : [get_active_users(date)] for date in date_set}
df3 = {date : [len(get_active_users(date)),len(get_active_users(date) - get_all_prior_users(date)), len(get_all_prior_users(date).intersection(get_active_users(date) - get_active_users(date-datetime.timedelta(days=1)))), len(get_active_users(date-datetime.timedelta(days=1)) -get_active_users(date))] for date in date_set}
ans1 = pd.DataFrame.from_dict(df3,'index', columns=['Active Users', 'New Users', 'Resurrected Users', 'Churned Users' ])

In [4]:
from itertools import product as pr
df4 = pd.merge(pd.DataFrame((i for i in pr(user_list,date_set)),columns=['user','date']),df,on='user',how='outer')
df4['diff'] = df4['date_x']-df4['date_y']


In [5]:
df5 = df4[df4['diff'].map(lambda x: x.days) >= 0]

In [23]:
def myfun(series):
    s4 = series['diff'].map(lambda x: x.days)
    s0 = s4[s4 >= 0].sort_values()
    if s0.empty:
        return 'NonVal'
    if s0.iloc[0]==0 and len(s0)==1:
        val = 'New'
    elif s0.iloc[0]==0 and s0.iloc[1] >= 28:
        val = 'Returning'
    elif s0.iloc[0] < 28 and s0.iloc[0]>=0:
        val = 'Active'
    elif s0.iloc[0] == 28:
        val = 'Churned'
    else:
        val = 'lost'
    return val

In [41]:
df6 = df5.groupby(['user', 'date_x']).apply(myfun)
df7 = pd.DataFrame(df6).reset_index().sort_values(by='date_x').groupby(['date_x',0]).count().unstack()
df7 = df7.fillna(0)
df7['user','Active'] = df7['user','Active'] + df7['user','New']

In [43]:
df7.head(140)

user                              
0          Active Churned  New Returning   lost
date_x                                         
2018-01-01    3.0     0.0  3.0       0.0    0.0
2018-01-02    8.0     0.0  5.0       0.0    0.0
2018-01-03   11.0     0.0  3.0       0.0    0.0
2018-01-04   13.0     0.0  2.0       0.0    0.0
2018-01-05   16.0     0.0  3.0       0.0    0.0
2018-01-06   19.0     0.0  3.0       0.0    0.0
2018-01-07   25.0     0.0  6.0       0.0    0.0
2018-01-08   31.0     0.0  6.0       0.0    0.0
2018-01-09   36.0     0.0  5.0       0.0    0.0
2018-01-10   40.0     0.0  4.0       0.0    0.0
2018-01-11   41.0     0.0  1.0       0.0    0.0
2018-01-12   44.0     0.0  3.0       0.0    0.0
2018-01-13   47.0     0.0  3.0       0.0    0.0
2018-01-14   48.0     0.0  1.0       0.0    0.0
2018-01-15   49.0     0.0  1.0       0.0    0.0
2018-01-16   51.0     0.0  2.0       0.0    0.0
2018-01-17   55.0     0.0  4.0       0.0    0.0
2018-01-18   57.0     0.0  2.0       0.0    0.0
2018-01-19   61.0     0.0  4.0       0.0    0.0
2018-01-20   62.0     0.0  1.0       0.0    0.0
2018-01-21   64.0     0.0  2.0       0.0    0.0
2018-01-22   66.0     0.0  2.0       0.0    0.0
2018-01-23   71.0     0.0  5.0       0.0    0.0
2018-01-24   73.0     0.0  2.0       0.0    0.0
2018-01-25   76.0     0.0  3.0       0.0    0.0
2018-01-26   77.0     0.0  1.0       0.0    0.0
2018-01-27   79.0     0.0  2.0       0.0    0.0
2018-01-28   80.0     0.0  1.0       0.0    0.0
2018-01-29   79.0     2.0  1.0       0.0    0.0
2018-01-30   76.0     4.0  1.0       0.0    2.0
...           ...     ...  ...       ...    ...
2018-04-21   71.0     2.0  0.0       3.0   95.0
2018-04-22   73.0     1.0  0.0       1.0   96.0
2018-04-23   73.0     2.0  1.0       1.0   96.0
2018-04-24   74.0     1.0  1.0       4.0   94.0
2018-04-25   76.0     2.0  0.0       1.0   94.0
2018-04-26   75.0     2.0  0.0       4.0   92.0
2018-04-27   77.0     2.0  0.0       2.0   92.0
2018-04-28   76.0     4.0  1.0       1.0   93.0
2018-04-29   76.0     2.0  1.0       2.0   95.0
2018-04-30   76.0     3.0  1.0       0.0   97.0
2018-05-01   73.0     3.0  0.0       2.0   98.0
2018-05-02   74.0     1.0  0.0       1.0  100.0
2018-05-03   74.0     1.0  0.0       1.0  100.0
2018-05-04   72.0     3.0  0.0       2.0   99.0
2018-05-05   76.0     0.0  2.0       4.0   98.0
2018-05-06   75.0     5.0  0.0       1.0   97.0
2018-05-07   74.0     3.0  1.0       1.0  101.0
2018-05-08   76.0     0.0  1.0       3.0  101.0
2018-05-09   77.0     2.0  0.0       1.0  100.0
2018-05-10   77.0     1.0  0.0       5.0   97.0
2018-05-11   81.0     1.0  0.0       1.0   97.0
2018-05-12   82.0     1.0  1.0       2.0   96.0
2018-05-13   79.0     4.0  0.0       3.0   95.0
2018-05-14   79.0     2.0  0.0       1.0   99.0
2018-05-15   80.0     1.0  1.0       1.0  100.0
2018-05-16   78.0     3.0  0.0       2.0   99.0
2018-05-17   76.0     4.0  0.0       1.0  101.0
2018-05-18   76.0     1.0  0.0       1.0  104.0
2018-05-19   74.0     3.0  0.0       1.0  104.0
2018-05-20   74.0     1.0  0.0       2.0  105.0

[140 rows x 5 columns]

In [44]:
ans1.head(140)

,Active Users,New Users,Resurrected Users,Churned Users
2018-01-01,3,3,0,0
2018-01-02,8,5,0,0
2018-01-03,11,3,0,0
2018-01-04,13,2,0,0
2018-01-05,16,3,0,0
2018-01-06,19,3,0,0
2018-01-07,25,6,0,0
2018-01-08,31,6,0,0
2018-01-09,36,5,0,0
2018-01-10,40,4,0,0


In [9]:
ans1

,Active Users,New Users,Resurrected Users,Churned Users
2018-01-01,3,3,0,0
2018-01-02,8,5,0,0
2018-01-03,11,3,0,0
2018-01-04,13,2,0,0
2018-01-05,16,3,0,0
2018-01-06,19,3,0,0
2018-01-07,25,6,0,0
2018-01-08,31,6,0,0
2018-01-09,36,5,0,0
2018-01-10,40,4,0,0


In [200]:
df.set_index('date',inplace = True)

In [168]:
srt = set(df4['diff'].sort_values(ascending=False).map(lambda x: x.days))

In [176]:
sr[sr>0]

636434     279.0
1376197    279.0
250597     279.0
1071276    279.0
773356     279.0
159314     279.0
527517     279.0
1273719    279.0
711196     279.0
1368357    279.0
403477     279.0
783436     279.0
678438     279.0
799676     279.0
129358     279.0
948079     279.0
1402518    279.0
52357      279.0
1106558    279.0
14278      279.0
1041598    279.0
1396357    279.0
693830     279.0
139157     278.0
20715      278.0
52354      278.0
168838     278.0
799672     278.0
1201196    278.0
250594     278.0
           ...  
1020354      1.0
1086932      1.0
1312724      1.0
919541       1.0
1098330      1.0
330796       1.0
520649       1.0
332038       1.0
461594       1.0
537401       1.0
1097700      1.0
298566       1.0
332305       1.0
461679       1.0
332049       1.0
98298        1.0
78183        1.0
891344       1.0
1087822      1.0
1381792      1.0
519999       1.0
521871       1.0
789595       1.0
1086490      1.0
1020144      1.0
201187       1.0
991627       1.0
1248241      1

In [252]:
sr.iloc[0]

279.0